# [How do I ask a good question?](http://)
### When someone tells me about an issue they are facing in development, in most cases, my immediate response would be: “What did you find on Stack Overflow?”
### And, there are people who give weird answers like: “I don’t know how to use it, I have been banned for asking poor questions, people always downvote my posts or give me links on how to ask questions,” etc.
![](https://miro.medium.com/max/1050/1*IF-ngtmrMVZGKLrvfEEf-w.png)


### Stack Overflow is one of the most helpful and most visited websites on the internet, but it is also one of the most brutal platforms on the internet. People will show no mercy if you make a mistake or ask a stupid question, and that’s how the platform has maintained its standards ever since its inception. So, don’t expect anything less.
### Instead, get used to it and change the way you ask questions. Enough said, let’s see what some of the important things are that you have to take care of while making a post on Stackoverflow.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from collections import Counter

from plotly import graph_objs as go
from sklearn import preprocessing 
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout, Bidirectional, Conv2D
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import transformers
from tokenizers import BertWordPieceTokenizer
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
import transformers

## [BRIEF DESCRIPTION OF DATASET:](http://)
This is an original dataset, made publicly available for researchers.
We collected 60,000 Stack Overflow questions from 2016-2020 and classified them into three categories:
1. **HQ**: High-quality posts with 30+ score and without a single edit.
3. **LQ_EDIT**: Low-quality posts with a negative score and with multiple community edits. However, they still remain open after the edits.
3. **LQ_CLOSE**: Low-quality posts that were closed by the community without a single edit.

In [ ]:
df = pd.read_csv("../input/60k-stack-overflow-questions-with-quality-rate/data.csv")
df.head()

In [ ]:
df.columns

In [ ]:
# Adding the title and body of query
df['text'] = df['Title'] + " " + df['Body']

# Drop columns not used for modelling
cols_to_drop = ['Id', 'Tags', 'CreationDate', 'Title', 'Body']
df.drop(cols_to_drop, axis=1, inplace=True)

# Rename category column to be more meaningful
df = df.rename(columns={"Y": "class"})

print("Total number of samples:", len(df))

df.head()

## [DATA VISUALIZATION AND PREPROCESSING](http://)

In [ ]:
temp = df.groupby('class').count()['text'].reset_index()

fig = go.Figure(go.Funnelarea(
    text = temp['class'],
    values = temp.text,
    title = {"position" : "top center", "text" : "Funnel Chart for target distribution"}
    ))
fig.show()

In [ ]:
temp = df['class'].value_counts()

fig = px.bar(temp)
fig.update_layout(
    title_text='Data distribution for each category',
    yaxis=dict(
        title='Count'
    ),
    xaxis=dict(
        title='Label'
    )
)
fig.show()

### SO, WE CAN SEE THAT THE DATASET IS BALANCED

In [ ]:
high = df[df['class']=='HQ']['text'].str.split().map(lambda x: len(x) if len(x) < 500 else 500)
low_open = df[df['class']=='LQ_EDIT']['text'].str.split().map(lambda x: len(x) if len(x) < 500 else 500)
low_closed = df[df['class']=='LQ_CLOSE']['text'].str.split().map(lambda x: len(x) if len(x) < 500 else 500)

fig = go.Figure()
fig.add_trace(go.Histogram(x=high, histfunc='avg', name="HQ", opacity=0.6, histnorm='probability density'))
fig.add_trace(go.Histogram(x=low_open, histfunc='avg', name="LQ_EDIT", opacity=0.6, histnorm='probability density'))
fig.add_trace(go.Histogram(x=low_closed, histfunc='avg', name="LQ_CLOSE", opacity=0.6, histnorm='probability density'))

fig.update_layout(
    title_text='Number of words in post', # title of plot
    xaxis_title_text='Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2,
    bargroupgap=0.1,
    barmode='overlay'
)
fig.show()

### The three histrograms show cleary, that high quality posts are generally longer than low quality posts.

In [ ]:
df.isna().sum() # Checking for nan Values

### [WHAT ARE STOPWORDS?](http://)

**Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Such words are already captured this in corpus named corpus. We first download it to our python environment.**

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

## [DATA CLEANING:](http://)

In [ ]:
# Clean the data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^(a-zA-Z)\s]','', text)
    return text
df['text'] = df['text'].apply(clean_text)

### [WORDCLOUD FOR HIGH QUALITY POSTS:](http://)

In [ ]:
plt.figure(figsize = (20,20)) # Text that is of high quality
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df[df['class'] == 'HQ'].text))
plt.imshow(wc , interpolation = 'bilinear')

### [WORDCLOUD FOR LOW QUALITY POSTS(CLOSED):](http://)

In [ ]:
plt.figure(figsize = (20,20)) # Text that is of low quality(closed)
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df[df['class'] == 'LQ_EDIT'].text))
plt.imshow(wc , interpolation = 'bilinear')

### [WORDCLOUD FOR LOW QUALITY POSTS(OPEN):](http://)

In [ ]:
plt.figure(figsize = (20,20)) # Text that is of low quality(open)
wc = WordCloud(max_words = 1000 , width = 1400 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df[df['class'] == 'LQ_CLOSE'].text))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'class'. 
df['class']= label_encoder.fit_transform(df['class']) 
  
df['class'].unique() 

### [COMMON WORDS IN HIGH QUALITY POSTS:](http://)

In [ ]:
df['temp_list'] = df['text'].apply(lambda x:str(x).split())

top = Counter([item for sublist in df['temp_list'].loc[df['class'] == 0] for item in sublist])
top_hq = pd.DataFrame(top.most_common(15))
top_hq.columns = ['Common_words','count']

fig = px.bar(top_hq, x='count',y='Common_words',title='Common words in High Quality posts',orientation='h',width=700,height=500,color='Common_words')
fig.show()

fig = px.treemap(top_hq, path=['Common_words'], values='count',title='Tree of Common words in High Quality posts')
fig.show()

In [ ]:
fig = px.pie(top_hq,
             values='count',
             names='Common_words',
             title='Word distribution in High Quality posts')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### [COMMON WORDS IN LOW QUALITY POSTS(CLOSED):](http://)

In [ ]:
top = Counter([item for sublist in df['temp_list'].loc[df['class'] == 1] for item in sublist])
top_lq = pd.DataFrame(top.most_common(15))
top_lq.columns = ['Common_words','count']

fig = px.bar(top_lq, x='count',y='Common_words',title='Common words in Low Quality posts(Closed)',orientation='h',width=700,height=500,color='Common_words')
fig.show()

fig = px.treemap(top_lq, path=['Common_words'], values='count',title='Tree of Common words in Low Quality posts')
fig.show()

In [ ]:
fig = px.pie(top_lq,
             values='count',
             names='Common_words',
             title='Word distribution in Low Quality posts(Closed)')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### [COMMON WORDS IN LOW QUALITY POSTS(OPEN):](http://)

In [ ]:
top = Counter([item for sublist in df['temp_list'].loc[df['class'] == 2] for item in sublist])
top_lq = pd.DataFrame(top.most_common(15))
top_lq.columns = ['Common_words','count']

fig = px.bar(top_lq, x='count',y='Common_words',title='Common words in Low Quality posts(Open)',orientation='h',width=700,height=500,color='Common_words')
fig.show()

fig = px.treemap(top_lq, path=['Common_words'], values='count',title='Tree of Common words in Low Quality posts')
fig.show()

In [ ]:
del df['temp_list']

In [ ]:
fig = px.pie(top_lq,
             values='count',
             names='Common_words',
             title='Word distribution in Low Quality posts(Open)')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
df.head()

### [NUMBER OF CHARACTERS IN POSTS:](http://)

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(16,6))
text_len=df[df['class']==0]['text'].str.len()
ax1.hist(text_len,color='lightcoral')
ax1.set_title('High Quality')
text_len=df[df['class']==1]['text'].str.len()
ax2.hist(text_len,color='lightgreen')
ax2.set_title('Low Quality(closed)')
text_len=df[df['class']==2]['text'].str.len()
ax3.hist(text_len,color='lightskyblue')
ax3.set_title('Low Quality(open)')
fig.suptitle('Characters in texts')
plt.show()

### [AVERAGE WORD-LENGTH IN POST:](http://)

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(16,6))
word=df[df['class']==0]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax1,color='lightcoral')
ax1.set_title('High Quality')
word=df[df['class']==1]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax2,color='lightgreen')
ax2.set_title('Low Quality(closed)')
word=df[df['class']==2]['text'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax3,color='lightskyblue')
ax3.set_title('Low Quality(open)')
fig.suptitle('Average word length in each text')

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

### [UNIGRAM ANALYSIS:](http://)

In [ ]:
plt.figure(figsize = (16,5))
most_common_uni = get_top_text_ngrams(df.text,10,1)
most_common_uni = dict(most_common_uni)
sns.set_palette("husl")
sns.barplot(x=list(most_common_uni.values()),y=list(most_common_uni.keys()))

### [BIGRAM ANALYSIS](http://)

In [ ]:
plt.figure(figsize = (16,5))
most_common_bi = get_top_text_ngrams(df.text,10,2)
most_common_bi = dict(most_common_bi)
sns.set_palette("husl")
sns.barplot(x=list(most_common_bi.values()),y=list(most_common_bi.keys()))

### [TRIGRAM ANALYSIS](http://)

In [ ]:
plt.figure(figsize = (16,5))
most_common_tri = get_top_text_ngrams(df.text,10,3)
most_common_tri = dict(most_common_tri)
sns.set_palette("husl")
sns.barplot(x=list(most_common_tri.values()),y=list(most_common_tri.keys()))

In [ ]:
df.head()

**Splitting the data into 2 parts - training and testing data**

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df['text'], df['class'], test_size = 0.2, random_state = 42, stratify = df['class'])

In [ ]:
max_features = 10000
maxlen = 300

**Tokenizing Text -> Repsesenting each word by a number**

**Mapping of orginal word to number is preserved in word_index property of tokenizer**

**Tokenized applies basic processing like changing it to lower case, explicitely setting that as False**

**Lets keep all news to 300, add padding to news with less than 300 words and truncating long ones**

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

## [Introduction to GloVe](http://)
**GloVe method is built on an important idea, You can derive semantic relationships between words from the co-occurrence matrix. Given a corpus having V words, the co-occurrence matrix X will be a V x V matrix, where the i th row and j th column of X, X_ij denotes how many times word i has co-occurred with word j. An example co-occurrence matrix might look as follows.**
![](https://miro.medium.com/max/521/1*QWcK8CIDs8kMkOwsOxvywA.png)
**The co-occurrence matrix for the sentence “the cat sat on the mat” with a window size of 1. As you probably noticed it is a symmetric matrix. How do we get a metric that measures semantic similarity between words from this? For that, you will need three words at a time. Let me concretely lay down this statement.**
![](https://miro.medium.com/max/860/1*8EI7ODLsxIX9hr7gEJnLSw.png)
**The behavior of P_ik/P_jk for various words Consider the entity P_ik/P_jk where P_ik = X_ik/X_i Here P_ik denotes the probability of seeing word i and k together, which is computed by dividing the number of times i and k appeared together (X_ik) by the total number of times word i appeared in the corpus (X_i). You can see that given two words, i.e. ice and steam, if the third word k (also called the “probe word”), is very similar to ice but irrelevant to steam (e.g. k=solid), P_ik/P_jk will be very high (>1), is very similar to steam but irrelevant to ice (e.g. k=gas), P_ik/P_jk will be very small (<1), is related or unrelated to either words, then P_ik/P_jk will be close to 1 So, if we can find a way to incorporate P_ik/P_jk to computing word vectors we will be achieving the goal of using global statistics when learning word vectors.**

**Source Credits - https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010**

In [ ]:
EMBEDDING_FILE = '../input/glove-twitter/glove.twitter.27B.200d.txt'

In [ ]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
batch_size = 256
epochs = 5
embed_size = 200

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.4, min_lr=0.0000001)

### [TRAINING THE MODEL](http://)

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    #Defining Neural Network
    model = Sequential()
    #Non-trainable embeddidng layer
    model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=True))
    #LSTM 
    model.add(Bidirectional(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.4 , dropout = 0.4)))
    model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.2 , dropout = 0.2)))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(lr = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

### [ANALYSIS AFTER TRAINING OF MODEL](http://)

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train,y_train)[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

In [ ]:
epochs = [i for i in range(5)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'go-' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'ro-' , label = 'Testing Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
pred = model.predict_classes(X_test)

In [ ]:
print(classification_report(y_test, pred, target_names = ['HQ', 'LQ(Close)', 'LQ(Open)']))

In [ ]:
cm = confusion_matrix(y_test,pred)
cm

In [ ]:
cm = pd.DataFrame(cm , index = ['HQ', 'LQ(Close)', 'LQ(Open)'] , columns = ['HQ', 'LQ(Close)', 'LQ(Open)'])

plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['HQ', 'LQ(Close)', 'LQ(Open)'] , yticklabels = ['HQ', 'LQ(Close)', 'LQ(Open)'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

## [Introduction to BERT](http://)
**BERT makes use of Transformer, an attention mechanism that learns contextual relations between words (or sub-words) in a text. In its vanilla form, Transformer includes two separate mechanisms — an encoder that reads the text input and a decoder that produces a prediction for the task. Since BERT’s goal is to generate a language model, only the encoder mechanism is necessary.**

**The chart below is a high-level description of the Transformer encoder. The input is a sequence of tokens, which are first embedded into vectors and then processed in the neural network. The output is a sequence of vectors of size H, in which each vector corresponds to an input token with the same index.**
![](https://miro.medium.com/max/1050/0*ViwaI3Vvbnd-CJSQ.png)
**As opposed to directional models, which read the text input sequentially (left-to-right or right-to-left), the Transformer encoder reads the entire sequence of words at once. Therefore it is considered bidirectional, though it would be more accurate to say that it’s non-directional. This characteristic allows the model to learn the context of a word based on all of its surroundings (left and right of the word).**

**Source Credits - https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270**

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df['text'], df['class'], random_state = 0 , stratify = df['class'])

In [ ]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=200):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
x_train = fast_encode(x_train.values, fast_tokenizer, maxlen=200)
x_test = fast_encode(x_test.values, fast_tokenizer, maxlen=200)

In [ ]:
# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    max_len=200
    transformer =  transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(3, activation='softmax')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(optimizer=keras.optimizers.Adam(lr=7e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train,y_train,batch_size = 64 ,validation_data=(x_test,y_test),epochs = 5)

In [ ]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [ ]:
epochs = [i for i in range(5)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'go-' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'ro-' , label = 'Testing Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
pred = model.predict(x_test)

In [ ]:
cm = pd.DataFrame(cm , index = ['HQ', 'LQ(Close)', 'LQ(Open)'] , columns = ['HQ', 'LQ(Close)', 'LQ(Open)'])

plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['HQ', 'LQ(Close)', 'LQ(Open)'] , yticklabels = ['HQ', 'LQ(Close)', 'LQ(Open)'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

## [PLS UPVOTE THIS NOTEBOOK IF YOU LIKE IT! THANKS FOR YOUR TIME !](http://)